## Code from Microsoft Site 
#### https://docs.microsoft.com/en-us/microsoft-365/enterprise/microsoft-365-ip-web-service?view=o365-worldwide
#### IP Addresses are Sorted Correctly

In [1]:
import json
from pathlib import Path
import urllib.request
import uuid
import ipaddress


In [2]:
# helper to call the webservice and parse the response
def webApiGet(methodName, instanceName, clientRequestId):
    ws = "https://endpoints.office.com"
    requestPath = f"{ws}/{methodName}/{instanceName}?clientRequestId={clientRequestId}"
    request = urllib.request.Request(requestPath)
    with urllib.request.urlopen(request) as response:
        return json.loads(response.read().decode())


In [3]:
datapath = Path('1.1 Microsoft URLs and IPs latestversion.txt')
# fetch client ID and version if data exists; otherwise create new file
if datapath.exists():
    with open(datapath, 'r') as fin:
        clientRequestId = fin.readline().strip()
        latestVersion = fin.readline().strip()
else:
    clientRequestId = str(uuid.uuid4())
    latestVersion = '0000000000'
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + latestVersion)


In [4]:
# call version method to check the latest version, and pull new data if version number is different

version = webApiGet('version', 'Worldwide', clientRequestId)

if version['latest'] > latestVersion:
    print('\nNew version of Office 365 worldwide commercial service instance endpoints detected')

    # write the new version number to the data file
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + version['latest'])

    # invoke endpoints method to get the new data
    endpointSets = webApiGet('endpoints', 'Worldwide', clientRequestId)

    flatIps = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow'):
            ips = endpointSet['ips'] if 'ips' in endpointSet else []
            ip4s = [ip for ip in ips if '.' in ip]
            flatIps.extend([ip for ip in ip4s])

    print('\nIPv4 Firewall IP Address Ranges')
    print(', '.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))                        # IP Address Sort

    with open('1.2 Microsoft IPs.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))       # IP Address Sort

    flatUrls = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow'):
            urls = endpointSet['urls'] if 'urls' in endpointSet else []
            flatUrls.extend([url for url in urls])

    print('\nURLs for Proxy Server')
    print(','.join(sorted(set(flatUrls))))

    with open('1.3 Microsoft URLs.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatUrls))))

else:
    print('Office 365 worldwide commercial service instance endpoints are up-to-date')



New version of Office 365 worldwide commercial service instance endpoints detected

IPv4 Firewall IP Address Ranges
13.107.6.152/31, 13.107.6.171/32, 13.107.18.10/31, 13.107.18.15/32, 13.107.64.0/18, 13.107.128.0/22, 13.107.136.0/22, 13.107.140.6/32, 20.190.128.0/18, 23.103.160.0/20, 40.92.0.0/15, 40.96.0.0/13, 40.104.0.0/15, 40.107.0.0/16, 40.108.128.0/17, 40.126.0.0/18, 52.96.0.0/14, 52.100.0.0/14, 52.104.0.0/14, 52.108.0.0/14, 52.112.0.0/14, 52.120.0.0/14, 52.238.78.88/32, 52.238.106.116/32, 52.238.119.141/32, 52.244.37.168/32, 52.244.160.207/32, 52.244.203.72/32, 52.244.207.172/32, 52.244.223.198/32, 52.247.150.191/32, 104.47.0.0/17, 104.146.128.0/17, 131.253.33.215/32, 132.245.0.0/16, 150.171.32.0/22, 150.171.40.0/22, 204.79.197.215/32

URLs for Proxy Server
*.auth.microsoft.com,*.broadcast.skype.com,*.compliance.microsoft.com,*.lync.com,*.mail.protection.outlook.com,*.msftidentity.com,*.msidentity.com,*.officeapps.live.com,*.online.office.com,*.outlook.office.com,*.protection.of